In [1]:
from core import Seed, init

init()

In [11]:
# optionally, help find the URL for the seed
from google_search import search

for result in search("Kevala"):
    print(result)

title='Kevala: The Future of Healthcare Staffing' link='https://www.kevala.care/' snippet='Kevala brings together your entire workforce behind a single login, allowing your schedulers the unprecedented ability to balance shift coverage, cost-\xa0...' formattedUrl='https://www.kevala.care/'
title='Kevala | Grid Intelligence Delivered' link='https://www.kevala.com/' snippet='Kevala equips energy leaders with advanced grid analytics to seamlessly integrate renewables, accurately predict electrification impacts, and enhance grid\xa0...' formattedUrl='https://www.kevala.com/'
title='Kevala.net: Organic products – Kevala' link='https://kevala.net/' snippet='At Kevala, our passion is to find unique, organic, high-quality ingredients, bringing the best for our valued customers.' formattedUrl='https://kevala.net/'
title='Kevala Wellness and Ayurveda: Home' link='https://kevalawellness.com/' snippet='Our center offers a range of Ayurvedic services designed to rejuvenate, restore, and sustain you

In [8]:
from datetime import datetime, timedelta
import os
from typing import Optional

target = Seed.init("Imagine Pediatrics", domain="imaginepediatrics.org")
data_folder = "../output/data"

output_json = f"{data_folder}/{target.as_path_v2()}.json"
output_json

def get_file_age(file_path: str) -> Optional[timedelta]:
    if os.path.exists(file_path):
        file_mod_time = datetime.fromtimestamp(os.path.getmtime(file_path))
        return datetime.now() - file_mod_time
    else:
        return None
    
def should_rebuild(file_path: str, max_age: timedelta = timedelta(days=7)) -> bool:
    age = get_file_age(file_path)
    if age is None:
        return True
    return age > max_age

should_rebuild(output_json)


True

In [9]:
# Set the log level
import sys
from loguru import logger
logger.remove()
logger.add(sys.stderr, level="INFO")

import unified

if not should_rebuild(output_json):
    print(f"Skipping {output_json} as it is up to date.")
else:
    unified_result = await unified.run(
        target, 
        num_reddit_threads=10, 
        max_glassdoor_review_pages=5, 
        max_glassdoor_job_pages=0,
        max_news_articles=20,
        )
    
    with open(output_json, 'w') as json_file:
        json_file.write(unified_result.model_dump_json(indent=2))



2024-09-30 11:44:56.263 | INFO     | company_webpage:run:66 - 66,774 -> 3,865 chars (6%) 
2024-09-30 11:44:56.271 | INFO     | company_webpage:run:66 - Extractive fraction: 17% 
2024-09-30 11:44:56.272 | INFO     | company_webpage:run:66 - Percent of URLs in sources: 94% ❌
2024-09-30 11:44:56.272 | INFO     | company_webpage:run:66 - Citation density: 19.8% (percent of output used by URLs/link syntax) 
2024-09-30 11:44:56.273 | INFO     | company_webpage:run:66 - Distinct URLs (summary / input): 16 / 29
2024-09-30 11:44:56.273 | INFO     | company_webpage:run:66 - Suspicious URLs: {'https://www.imaginepediatrics.org/events/'}
2024-09-30 11:44:56.274 | INFO     | company_webpage:run:66 - Cache mentions: 0 ✅
2024-09-30 11:44:58.790 | INFO     | core:shorten_markdown:228 - 13,763 -> 11,458 chars (83% of original)
2024-09-30 11:45:05.517 | INFO     | core:unshorten_markdown:247 - 2,134 -> 3,307 chars (155% of original)
2024-09-30 11:45:05.518 | INFO     | general_search:summarize:126 - 13,